In [1]:
#1 Danny Severe

In [2]:
#2

import pandas as pd
import psycopg2 as psy
import warnings

warnings.filterwarnings("ignore")

conn = psy.connect(database='corgirace', \
                   user='dannysevere', \
                   password='Severe21!', \
                   host='dbsevere.ct4e9f3blg2v.us-west-1.rds.amazonaws.com', \
                   port='5432')

cursor = conn.cursor()

query = 'SELECT * FROM CORGI'

df = pd.read_sql(query, conn, index_col='corgid')
print(df)

                         corgname  breed gender  weight  age fromfile
corgid                                                               
1                         Brookie    Pem      F      21  3.5     None
2                   Golden Graham    Pem      M      30  2.0     None
3                          Kraken    Pem     SF      22  3.5     None
4                     Snarla June    Pem     SF      29  3.5     None
5                           Isaac  Cardi     NM      33  4.5     None
6                         Stanley    Pem      M      33  3.5     None
7                         Optimus    Pem      M      25  4.0     None
8                        Gu∂ri∂ur    Pem      F      20  5.0     None
9       Baroness von Fluffyshanks    Pem     SF      22  4.0     None
10                     Prometheus    Pem      M      27  5.0     None
11                 Foxy Stumptail    Pem     NM      29  5.5     None
12                      Drederick    Pem      M      29  4.5     None
13              Lady

In [10]:
#3

print("Corgname: Should be VarChar(30) to accomodate long names")
print("Breed: The only valid inputs would be Pem and Cardi.")
print("Gender: Only valid inputs are F, M, SF, and NM.")
print("Weight: Should be two digit values ranging from 10-40.")
print("Age: Age will be a two digit value with one digit after the decimal.")

Corgname: Should be VarChar(30) to accomodate long names
Breed: The only valid inputs would be Pem and Cardi.
Gender: Only valid inputs are F, M, SF, and NM.
Weight: Should be two digit values ranging from 10-40.
Age: Age will be a two digit value with one digit after the decimal.


In [4]:
#4

query = '''CREATE TABLE IF NOT EXISTS ingest (
            Filename VARCHAR(50) PRIMARY KEY,
            Whendone TIMESTAMP NOT NULL);'''

cursor.execute(query)
conn.commit()


In [5]:
#5

query = '''ALTER TABLE CORGI
            ADD COLUMN IF NOT EXISTS fromfile VARCHAR(50);'''

cursor.execute(query)
conn.commit()

In [6]:
query = 'SELECT corgname, fromfile FROM CORGI'

df = pd.read_sql(query, conn)
print(df)

                     corgname fromfile
0                     Brookie     None
1               Golden Graham     None
2                      Kraken     None
3                 Snarla June     None
4                       Isaac     None
5                     Stanley     None
6                     Optimus     None
7                    Gu∂ri∂ur     None
8   Baroness von Fluffyshanks     None
9                  Prometheus     None
10             Foxy Stumptail     None
11                  Drederick     None
12          Lady Wigglebottom     None
13                   Jedediah     None
14               Miss Juniper     None
15        Mary Queen of Corgs     None
16                    Oatmeal     None
17                 Cowboy Dan     None
18                Gareth Bale     None
19              Winston Smith     None
20                      Bucco     None
21                   Fruitcup     None
22                  Sweet Dee     None
23                      Sempo     None
24                       

In [7]:
#6

csvname = 'Documents/python/newcorgis-20220618.csv'

query = '''
        SELECT COUNT(*)
        FROM CORGI
        WHERE fromfile = %s
        '''

cursor.execute(query, (csvname,))

y = cursor.fetchone()

if y[0] == True:
    print('This file has been processed')
    
if y[0] == False:
    print('This file has been accepted for processing')
    
    df2 = pd.read_csv(csvname)
    df2.rename(columns={'Name':'corgname', 'Breed':'breed', 'Gender':'gender', 'Weight':'weight', 'Age':'age'}, inplace=True)
    df2['breed'].replace(['PWC', 'CWC'], ['Pem', 'Cardi'], inplace=True)   
    df2['gender'].replace('Fem', 'F', inplace=True)
    df2['weight'] = df2['weight'].astype(int)
    print(df2)
    
    for x in df2.index:
        cursor.execute('''INSERT INTO CORGI (corgname, breed, gender, weight, age, fromfile)
                            VALUES ('%s', '%s', '%s', %s, %s, '%s')'''% (df2['corgname'].loc[x], df2['breed'].loc[x], df2['gender'].loc[x], df2['weight'].loc[x], df2['age'].loc[x], csvname))
        conn.commit()
        
        print('Record created for %s'% (df2['corgname'].loc[x]))
        
    cursor.execute('''INSERT INTO ingest (filename, whendone)
                            VALUES ('%s', current_timestamp)'''% (csvname))
    conn.commit()
    print('Ingestion is complete for', csvname)
    
    query = ('''SELECT *
            FROM ingest
            where filename = '%s'; '''% (csvname))

    df = pd.read_sql(query, conn)
    print(df)
    
    

This file has been accepted for processing
          corgname  breed gender  weight  age
0         Francine    Pem      F      24  3.5
1         Prudence    Pem     SF      29  4.0
2        Fort Knox  Cardi     NM      33  3.3
3          Gruyere    Pem     SF      22  1.5
4     Sir Barxalot    Pem      M      26  2.0
5      Tuning Fork  Cardi     NM      29  1.8
6          Wiffles    Pem     SF      23  5.0
7           Wumpus    Pem      F      26  1.2
8  Qween Kanonball    Pem      F      29  3.5
Record created for Francine
Record created for Prudence
Record created for Fort Knox
Record created for Gruyere
Record created for Sir Barxalot
Record created for Tuning Fork
Record created for Wiffles
Record created for Wumpus
Record created for Qween Kanonball
Ingestion is complete for Documents/python/newcorgis-20220618.csv
                                  filename                   whendone
0  Documents/python/newcorgis-20220618.csv 2024-02-24 02:53:44.509060


In [8]:
#7

query = ('''SELECT *
            FROM corgi
            where fromfile = '%s'; '''% (csvname))

dfv = pd.read_sql(query, conn)

print(dfv)

   corgid         corgname  breed gender  weight  age  \
0      26         Francine    Pem      F      24  3.5   
1      27         Prudence    Pem     SF      29  4.0   
2      28        Fort Knox  Cardi     NM      33  3.3   
3      29          Gruyere    Pem     SF      22  1.5   
4      30     Sir Barxalot    Pem      M      26  2.0   
5      31      Tuning Fork  Cardi     NM      29  1.8   
6      32          Wiffles    Pem     SF      23  5.0   
7      33           Wumpus    Pem      F      26  1.2   
8      34  Qween Kanonball    Pem      F      29  3.5   

                                  fromfile  
0  Documents/python/newcorgis-20220618.csv  
1  Documents/python/newcorgis-20220618.csv  
2  Documents/python/newcorgis-20220618.csv  
3  Documents/python/newcorgis-20220618.csv  
4  Documents/python/newcorgis-20220618.csv  
5  Documents/python/newcorgis-20220618.csv  
6  Documents/python/newcorgis-20220618.csv  
7  Documents/python/newcorgis-20220618.csv  
8  Documents/python/newc